In [6]:
df = spark.sql("SELECT * FROM bing_lake_database.latest_news")
display(df)

StatementMeta(, 0312d7a2-1ecd-4d56-9c91-09b72865c3ed, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 4e62505b-b604-4333-b44c-a672f791d8ae)

In [7]:
import synapse.ml.core
from synapse.ml.services import AnalyzeText

# Set up your Cognitive Services API key and endpoint
subscription_key = "******" #TODO
endpoint = "https://ai-lang-bing.openai.azure.com/" #TODO
location = 'eastus' #TODO

model = (AnalyzeText()
         .setTextCol('description')
         .setKind('SentimentAnalysis')
         .setOutputCol('response')
         .setErrorCol('error')
         .setSubscriptionKey(subscription_key)
         .setLocation(location)
         )

StatementMeta(, 0312d7a2-1ecd-4d56-9c91-09b72865c3ed, 9, Finished, Available, Finished)

In [8]:
result = model.transform(df)

StatementMeta(, 0312d7a2-1ecd-4d56-9c91-09b72865c3ed, 10, Finished, Available, Finished)

In [9]:
display(result)

StatementMeta(, 0312d7a2-1ecd-4d56-9c91-09b72865c3ed, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 30ef4516-bae0-46ce-ba81-b09885cb1335)

In [10]:
from pyspark.sql.functions import col

sentiment_df = result.withColumn('sentiment', col("response.documents.sentiment"))
display(sentiment_df)

StatementMeta(, 0312d7a2-1ecd-4d56-9c91-09b72865c3ed, 12, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, d04f3451-fe28-43ed-b8ae-02f4ed9f7b3e)

In [11]:
final_df = sentiment_df.drop('error', 'response')

from pyspark.sql.functions import col, to_date

final_df = final_df.withColumn('datePublished', to_date(col('datePublished'), 'dd-MMM-yyyy'))

StatementMeta(, 0312d7a2-1ecd-4d56-9c91-09b72865c3ed, 13, Finished, Available, Finished)

In [12]:
from pyspark.sql.utils import AnalysisException

try:
    table = 'bing_lake_database.sentiment_analysis'

    final_df.write.format("delta").saveAsTable(table)

except AnalysisException:
    print('Table already exists.')

    final_df.createOrReplaceTempView('temp_final_df')

    spark.sql(f"""  MERGE INTO {table} target_table
                    USING temp_final_df source_view

                    ON source_view.url = target_table.url

                    WHEN MATCHED AND
                    source_view.title <> target_table.title OR
                    source_view.description <> target_table.description OR
                    source_view.image <> target_table.image OR
                    source_view.url <> target_table.url OR
                    source_view.provider <> target_table.provider OR
                    source_view.datePublished <> target_table.datePublished

                    THEN UPDATE SET *

                    WHEN NOT MATCHED THEN INSERT *

                """)

StatementMeta(, 0312d7a2-1ecd-4d56-9c91-09b72865c3ed, 14, Finished, Available, Finished)